In [ ]:
from segmentation_models import Unet
from segmentation_models.backbones import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
import pickle
import preprocess
import splitdata
import os
import numpy as np

In [ ]:
with open(os.path.join(splitdata.d_data, splitdata.f_train), "rb") as file:
    files = pickle.load(file)
if not files:
    raise Exception("Could not load training files!")
print("Files: ", len(files))

np.random.shuffle(files)
n_valid = int(np.ceil(len(files) * 0.15))
files_train = files[n_valid:]
files_valid = files[:n_valid]
print("Files train: ", len(files_train))
print("Files valid: ", len(files_valid))

X_train, Y_train = preprocess.load_data(files_train)
X_valid, Y_valid = preprocess.load_data(files_valid)

In [ ]:
X_train = X_train[:,18:-18,18:-18,:] / 255
X_valid = X_valid[:,18:-18,18:-18,:] / 255
Y_train = Y_train[:,18:-18,18:-18,:]
Y_valid = Y_valid[:,18:-18,18:-18,:]

In [ ]:
preprocess_input = get_preprocessing('resnet34')

In [ ]:
model = Unet('resnet34', classes=2, input_shape=(2048, 2048, 3), activation='sigmoid')
model.compile('Adam', loss=bce_jaccard_loss, metrics=[iou_score])

In [ ]:
# fit model
model.fit(
    x=X_train,
    y=Y_train,
    batch_size=1,
    epochs=10,
    validation_data=(X_valid, Y_valid),
)